In [1]:
import os
import re
import sys
import json
from glob import glob
from collections import Counter, defaultdict
import torch
import pickle
import numpy as np
from tqdm import tqdm
from transformers import BertModel

2021-12-04 23:56:00.353127: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-04 23:56:00.353176: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
in_dir = os.path.join('data','preprocessed')
sample_file = os.path.join('data','preprocessed','sample_target_index.dict')
outdir = os.path.join('data','preprocessed')
batch_size = 200
layers = '10,11'

In [3]:
# collect index of tokens in the documents
files = sorted(glob(os.path.join(in_dir, '*_tokenized.jsonlist')))
docs = []
for file in files:
    with open(file) as f:
        docs.append(f.readlines())

In [4]:
# get sample index
with open(sample_file,'rb') as f:
    sample_index = pickle.load(f)

In [5]:
#layers
layers = [int(layer) for layer in layers.split(',')]

# load the model
model = BertModel.from_pretrained('bert-base-uncased')
# move the model to the GPU
if torch.cuda.is_available():
    model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def get_context(token_ids, target_position, sequence_length=128):
    """
    Given a text containing a target word, return the sentence snippet which surrounds the target word
    (and the target word's position in the snippet).
    :param token_ids: list of token ids (for an entire line of text)
    :param target_position: index of the target word's position in `tokens`
    :param sequence_length: desired length for output sequence (e.g. 128, 256, 512)
    :return: (context_ids, new_target_position)
                context_ids: list of token ids for the output sequence
                new_target_position: index of the target word's position in `context_ids`
    """
    # -2 as [CLS] and [SEP] tokens will be added later; /2 as it's a one-sided window
    window_size = int((sequence_length - 2) / 2)
    context_start = max([0, target_position - window_size])
    padding_offset = max([0, window_size - target_position])
    padding_offset += max([0, target_position + window_size - len(token_ids)])

    context_ids = token_ids[context_start:target_position + window_size]
    context_ids += padding_offset * [0]

    new_target_position = target_position - context_start

    return context_ids, new_target_position

In [9]:
# Just put the output into some lists for now
segment_id_list = []
token_list = []
piece_index_list = []
token_index_list = []
vectors_by_layer = defaultdict(list)
to_encode = []
encoded = []
targets = []
token_indices = []
lengths = []
line_ids = []

for target_word in sample_index:
    for file_id,doc_id,index in sample_index[target_word]:
        doc = json.loads(docs[file_id][doc_id])
        tokens = doc['tokens']
        # now we get the context
        context_ids, pos_in_context = get_context(tokens, index)
        input_ids = ['[CLS]']+context_ids+['[SEP]']
        encoded.append(input_ids)
        break
    break
        #if len(to_encode) == batch_size or (line_index == (n_lines-1) and len(to_encode) > 1):

In [10]:
encoded

[['[CLS]',
  ',',
  'sixteen',
  'others',
  ',',
  'spread',
  'over',
  'a',
  'country',
  'of',
  'two',
  'thousand',
  'miles',
  'diameter',
  ',',
  'rise',
  'up',
  'on',
  'every',
  'side',
  ',',
  'ready',
  'organised',
  ',',
  'for',
  'del',
  '##ibe',
  '##ration',
  'by',
  'a',
  'constitutional',
  'legislature',
  ',',
  '&',
  'for',
  'action',
  'by',
  'their',
  'governor',
  ',',
  'constitutional',
  '##ly',
  'the',
  'commander',
  'of',
  'the',
  'militia',
  'of',
  'the',
  'state',
  ',',
  'that',
  'is',
  'to',
  'say',
  ',',
  'of',
  'every',
  'man',
  'in',
  'it',
  ',',
  'able',
  'to',
  'bear',
  'arms',
  ';',
  'and',
  'that',
  'militia',
  'too',
  'regularly',
  'formed',
  'into',
  'regiments',
  '&',
  'battalions',
  ',',
  'into',
  'infantry',
  ',',
  'cavalry',
  '&',
  'artillery',
  ',',
  'trained',
  'under',
  'officers',
  'general',
  '&',
  'subordinate',
  ',',
  'legally',
  'appointed',
  ',',
  'always',
  'in'